In [1]:
import torch
import torchvision
from torch.utils.data import DataLoader, Dataset, random_split
from torch import nn
from torch.nn.utils import prune
import numpy as np
import sys
import matplotlib.pyplot as plt
import math
import random
import sklearn.metrics as perf
import os
import cv2
import time
import torch_pruning as tp
import csv
sys.path.insert(0, '..') 

from models.models import MTLClassifier, AgeRegressor, GenderClassifier, EthnicityClassifier
from utils.data import FacesDataset, data_transform
from utils.training import train_mtl_model, train_age_model, train_gender_model, train_ethnicity_model, train_mtl_model_individual
from utils.evaluation import run_evaluation, show_example_predictions
from utils.pruning import prune_model, prune_other_tasks, get_f1_and_lat

### Load and Prepare Data

In [2]:
### Load in the data
folder = '../UTKFace'
transform = data_transform()
dataset = FacesDataset(folder=folder, transform=transform)

In [3]:
### Set up train and val datasets and loaders
train_len = int(len(dataset)*0.8)
val_len = len(dataset) - train_len
train_dataset, val_dataset = random_split(dataset, [train_len, val_len], torch.Generator().manual_seed(8))

train_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=16, shuffle=False)

In [4]:
### Get small subset fpr train_loader and val_loader (for testing and debugging)
# train_indices = torch.randperm(len(train_dataset))[:100]
# val_indices = torch.randperm(len(val_dataset))[:100]
# train_subset = torch.utils.data.Subset(train_dataset, train_indices)
# val_subset = torch.utils.data.Subset(val_dataset, val_indices)
# train_loader = DataLoader(dataset=train_subset, batch_size=16, shuffle=True)
# val_loader = DataLoader(dataset=val_subset, batch_size=16, shuffle=True)

### MTL Model Variants

#### Define pruned training function

In [5]:
def pruned_mtl_training(task, prune_pct, num_epochs,
                        train_loader=train_loader, val_loader=val_loader,
                        val_dataset=None):
    
    ### Set up model, loss, and optimizer
    if task.upper()=='ALL':
        tasks = ['age', 'gender', 'ethnicity']
        model = MTLClassifier()
        model = model.cuda()
        age_criterion = nn.MSELoss()
        gender_criterion = nn.CrossEntropyLoss()
        ethnicity_criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters())

        # Set up and run model training
        age_coeff = 0.004
        gender_coeff = 2
        ethnicity_coeff = 1
        
        # Train initial model
        print('---------------- Train Initial Model ----------------')
        save_no_prune = f'model_variants/{task.lower()}_p-0_lat-init_f1-init_MTLModel.pth'
        train_mtl_model(num_epochs=num_epochs, model=model, optimizer=optimizer,
                        train_loader=train_loader, val_loader=val_loader,
                        age_criterion=age_criterion, gender_criterion=gender_criterion, ethnicity_criterion=ethnicity_criterion,
                        age_coeff=age_coeff, gender_coeff=gender_coeff, ethnicity_coeff=ethnicity_coeff, save=True,
                        save_name=save_no_prune)
        
        # Do pruning
        model = torch.load(f'models/{save_no_prune}')
        pruned_model = prune_model(model, PRUNING_PERCENT=prune_pct)
        pruned_optimizer = torch.optim.Adam(pruned_model.parameters())
        
        # Fine-tune model
        print('-------------- Fine-tuning Pruned Model -------------')
        save_initial = f'model_variants/{task.lower()}_p-{int(prune_pct*100)}_lat-init_f1-init_MTLModel.pth'
        train_mtl_model(num_epochs=num_epochs, model=pruned_model, optimizer=pruned_optimizer,
                        train_loader=train_loader, val_loader=val_loader,
                        age_criterion=age_criterion, gender_criterion=gender_criterion, ethnicity_criterion=ethnicity_criterion,
                        age_coeff=age_coeff, gender_coeff=gender_coeff, ethnicity_coeff=ethnicity_coeff, save=True,
                        save_name=save_initial)
        
        # Test latency and accuracy (F1) and save model variant (and update lookup file)
        [scores, [mean_lat, std_lat]] = get_f1_and_lat(model_path=save_initial,
                                                   eval_dataset=val_dataset,
                                                   eval_dataloader=val_loader,
                                                   tasks=tasks,
                                                   mtl_model=True)
        
        # Save model with score and latency information in the model name
        ager2 = scores['age'][1]
        genderf1 = scores['gender'][0]
        ethnicityf1 = scores['ethnicity'][0]
        row = [task.upper(), prune_pct, mean_lat, std_lat, ager2, genderf1, ethnicityf1]
        with open('models/model_variants/model_score_lookup_multitask.tsv', 'a', newline='') as f:
            writer = csv.writer(f, delimiter='\t')
            writer.writerow(row)
            
        new_name = f'model_variants/{task.lower()}_p-{int(prune_pct*100)}_MTLModel.pth'
        torch.save(model, f"models/{new_name}")
        
        
    elif task.upper()=='GENDER':
        tasks = ['gender']
        model = MTLClassifier()
        model = model.cuda()
        age_criterion = nn.MSELoss()
        gender_criterion = nn.CrossEntropyLoss()
        ethnicity_criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters())

        # Set up and run model training
        age_coeff = 0.004
        gender_coeff = 2
        ethnicity_coeff = 1
        
        # Train initial model
        print('---------------- Train Initial Model ----------------')
        save_no_prune = f'model_variants/{task.lower()}_p-0_lat-init_f1-init_MTLModel.pth'
        train_mtl_model(num_epochs=num_epochs, model=model, optimizer=optimizer,
                        train_loader=train_loader, val_loader=val_loader,
                        age_criterion=age_criterion, gender_criterion=gender_criterion, ethnicity_criterion=ethnicity_criterion,
                        age_coeff=age_coeff, gender_coeff=gender_coeff, ethnicity_coeff=ethnicity_coeff, save=True,
                        save_name=save_no_prune)
        
        # Do pruning
        model = torch.load(f'models/{save_no_prune}')
        pruned_model = prune_other_tasks(model, task1='age', task2='ethnicity', PRUNING_PERCENT=prune_pct)
        pruned_optimizer = torch.optim.Adam(pruned_model.parameters())
        
        # Fine-tune model
        print('-------------- Fine-tuning Pruned Model -------------')
        save_initial = f'model_variants/{task.lower()}_p-{int(prune_pct*100)}_lat-init_f1-init_MTLModel.pth'
        train_mtl_model_individual(num_epochs=num_epochs, model=pruned_model, optimizer=pruned_optimizer,
                              train_loader=train_loader, val_loader=val_loader,
                              gender_criterion=gender_criterion, age_coeff=1.0, gender_coeff=1.0, ethnicity_coeff=1.0,
                              save=True, save_name=save_initial, isGender=True)
        
        # Test latency and accuracy (F1) and save model variant (and update lookup file)
        [scores, [mean_lat, std_lat]] = get_f1_and_lat(model_path=save_initial,
                                                   eval_dataset=val_dataset,
                                                   eval_dataloader=val_loader,
                                                   tasks=tasks,
                                                   mtl_model=True)
        
        # Save model with score and latency information in the model name
        genderf1 = scores['gender'][0]
        row = [task.upper(), prune_pct, mean_lat, std_lat, 0.0, genderf1, 0.0]
        with open('models/model_variants/model_score_lookup_multitask.tsv', 'a', newline='') as f:
            writer = csv.writer(f, delimiter='\t')
            writer.writerow(row)
            
        new_name = f'model_variants/{task.lower()}_p-{int(prune_pct*100)}_MTLModel.pth'
        torch.save(model, f"models/{new_name}")
        
    elif task.upper()=='ETHNICITY':
        tasks = ['ethnicity']
        model = MTLClassifier()
        model = model.cuda()
        age_criterion = nn.MSELoss()
        gender_criterion = nn.CrossEntropyLoss()
        ethnicity_criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters())

        # Set up and run model training
        age_coeff = 0.004
        gender_coeff = 2
        ethnicity_coeff = 1
        
        
        # Train initial model
        print('---------------- Train Initial Model ----------------')
        save_no_prune = f'model_variants/{task.lower()}_p-0_lat-init_f1-init_MTLModel.pth'
        train_mtl_model(num_epochs=num_epochs, model=model, optimizer=optimizer,
                        train_loader=train_loader, val_loader=val_loader,
                        age_criterion=age_criterion, gender_criterion=gender_criterion, ethnicity_criterion=ethnicity_criterion,
                        age_coeff=age_coeff, gender_coeff=gender_coeff, ethnicity_coeff=ethnicity_coeff, save=True,
                        save_name=save_no_prune)
        
        # Do pruning
        model = torch.load(f'models/{save_no_prune}')
        pruned_model = prune_other_tasks(model, task1='age', task2='gender', PRUNING_PERCENT=prune_pct)
        pruned_optimizer = torch.optim.Adam(pruned_model.parameters())
        
        # Fine-tune model
        print('-------------- Fine-tuning Pruned Model -------------')
        save_initial = f'model_variants/{task.lower()}_p-{int(prune_pct*100)}_lat-init_f1-init_MTLModel.pth'
        train_mtl_model_individual(num_epochs=num_epochs, model=pruned_model, optimizer=pruned_optimizer,
                              train_loader=train_loader, val_loader=val_loader,
                              ethnicity_criterion=ethnicity_criterion, age_coeff=1.0, gender_coeff=1.0, ethnicity_coeff=1.0,
                              save=True, save_name=save_initial, isethnicity=True)
        
        # Test latency and accuracy (F1) and save model variant (and update lookup file)
        [scores, [mean_lat, std_lat]] = get_f1_and_lat(model_path=save_initial,
                                                       eval_dataset=val_dataset,
                                                       eval_dataloader=val_loader,
                                                       tasks=tasks,
                                                       mtl_model=True)
        
        # Save model with score and latency information in the model name
        ethnicityf1 = scores['ethnicity'][0]
        row = [task.upper(), prune_pct, mean_lat, std_lat, 0.0, 0.0, ethnicityf1]
        with open('models/model_variants/model_score_lookup_multitask.tsv', 'a', newline='') as f:
            writer = csv.writer(f, delimiter='\t')
            writer.writerow(row)
            
        new_name = f'model_variants/{task.lower()}_p-{int(prune_pct*100)}_MTLModel.pth'
        torch.save(model, f"models/{new_name}")
        
    elif task.upper()=='AGE':
        tasks = ['age']
        model = MTLClassifier()
        model = model.cuda()
        age_criterion = nn.MSELoss()
        gender_criterion = nn.CrossEntropyLoss()
        ethnicity_criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters())

        # Set up and run model training
        age_coeff = 0.004
        gender_coeff = 2
        ethnicity_coeff = 1
        
        
        # Train initial model
        print('---------------- Train Initial Model ----------------')
        save_no_prune = f'model_variants/{task.lower()}_p-0_lat-init_f1-init_MTLModel.pth'
        train_mtl_model(num_epochs=num_epochs, model=model, optimizer=optimizer,
                        train_loader=train_loader, val_loader=val_loader,
                        age_criterion=age_criterion, gender_criterion=gender_criterion, ethnicity_criterion=ethnicity_criterion,
                        age_coeff=age_coeff, gender_coeff=gender_coeff, ethnicity_coeff=ethnicity_coeff, save=True,
                        save_name=save_no_prune)
        
        # Do pruning
        model = torch.load(f'models/{save_no_prune}')
        pruned_model = prune_other_tasks(model, task1='gender', task2='ethnicity', PRUNING_PERCENT=prune_pct)
        pruned_optimizer = torch.optim.Adam(pruned_model.parameters())
        
        # Fine-tune model
        print('-------------- Fine-tuning Pruned Model -------------')
        save_initial = f'model_variants/{task.lower()}_p-{int(prune_pct*100)}_lat-init_f1-init_MTLModel.pth'
        train_mtl_model_individual(num_epochs=num_epochs, model=pruned_model, optimizer=pruned_optimizer,
                              train_loader=train_loader, val_loader=val_loader,
                              age_criterion=age_criterion, age_coeff=1.0, gender_coeff=1.0, ethnicity_coeff=1.0,
                              save=True, save_name=save_initial, isAge=True)

        
        # Test latency and accuracy (F1) and save model variant (and update lookup file)
        [scores, [mean_lat, std_lat]] = get_f1_and_lat(model_path=save_initial,
                                                       eval_dataset=val_dataset,
                                                       eval_dataloader=val_loader,
                                                       tasks=tasks,
                                                       mtl_model=True)
        
        # Save model with score and latency information in the model name
        ager2 = scores['age'][1]
        row = [task.upper(), prune_pct, mean_lat, std_lat, ager2, 0.0, 0.0]
        with open('models/model_variants/model_score_lookup_multitask.tsv', 'a', newline='') as f:
            writer = csv.writer(f, delimiter='\t')
            writer.writerow(row)
            
        new_name = f'model_variants/{task.lower()}_p-{int(prune_pct*100)}_MTLModel.pth'
        torch.save(model, f"models/{new_name}")
        
    else:
        print('Invalid task was specified.')
        
        
    # Delete intermediate model files
    os.remove(f'models/{save_initial}')
    if save_initial != save_no_prune:
        os.remove(f'models/{save_no_prune}')
        
    return scores, mean_lat, std_lat
        


#### 0% to 90% Pruning

In [6]:
task = 'all'
num_epochs = 10
with open('models/model_variants/model_score_lookup_multitask.tsv', 'w', newline='') as f:
    writer = csv.writer(f, delimiter='\t')
    writer.writerow(['Task', 'prune_pct', 'mean_latency', 'std_latency', 'age_r2', 'gender_f1', 'ethnicity_f1'])
            
for i in range(10):
    print(f'--------------------------------- Prune {i*10} % ---------------------------------')
    prune_pct = 1.0*i / 10
    scores, mean_lat, std_lat = pruned_mtl_training(task, prune_pct, num_epochs, train_loader, val_loader, val_dataset)
    print()

--------------------------------- Prune 0 % ---------------------------------
---------------- Train Initial Model ----------------
Epoch 0, val loss: inf -> 1.92090, train loss: 0.92761
Epoch 0, age val loss: 1.06885, gender val loss: 0.12269, ethnicity val loss: 0.72936
Epoch 1, val loss: 1.92090 -> 1.65414, train loss: 0.68848
Epoch 1, age val loss: 0.97036, gender val loss: 0.07513, ethnicity val loss: 0.60864
Epoch 2, val loss: 2.32135, train loss: 0.47244
Epoch 2, age val loss: 0.89243, gender val loss: 0.79133, ethnicity val loss: 0.63758
Epoch 3, val loss: 3.44443, train loss: 0.19925
Epoch 3, age val loss: 0.66216, gender val loss: 1.88251, ethnicity val loss: 0.89976
Epoch 4, val loss: 3.63189, train loss: 0.01368
Epoch 4, age val loss: 0.47690, gender val loss: 1.97096, ethnicity val loss: 1.18403
Epoch 5, val loss: 3.93709, train loss: 0.23983
Epoch 5, age val loss: 0.47694, gender val loss: 2.30057, ethnicity val loss: 1.15958
Epoch 6, val loss: 3.83549, train loss: 0.1504


--------------------------------- Prune 30 % ---------------------------------
---------------- Train Initial Model ----------------
Epoch 0, val loss: inf -> 2.11088, train loss: 0.92378
Epoch 0, age val loss: 1.08818, gender val loss: 0.45637, ethnicity val loss: 0.56632
Epoch 1, val loss: 2.11088 -> 1.89522, train loss: 0.71595
Epoch 1, age val loss: 0.96761, gender val loss: 0.10288, ethnicity val loss: 0.82473
Epoch 2, val loss: 2.24883, train loss: 0.47007
Epoch 2, age val loss: 0.77465, gender val loss: 0.00157, ethnicity val loss: 1.47261
Epoch 3, val loss: 3.43558, train loss: 0.21339
Epoch 3, age val loss: 0.64688, gender val loss: 0.00000, ethnicity val loss: 2.78870
Epoch 4, val loss: 4.69338, train loss: 0.08046
Epoch 4, age val loss: 1.05368, gender val loss: 0.00000, ethnicity val loss: 3.63970
Epoch 5, val loss: 3.96382, train loss: 0.13394
Epoch 5, age val loss: 0.72627, gender val loss: 0.00000, ethnicity val loss: 3.23756
Epoch 6, val loss: 3.94860, train loss: 0.11

Epoch 8, val loss: 4.43108, train loss: 0.37269
Epoch 8, age val loss: 0.82649, gender val loss: 0.00000, ethnicity val loss: 3.60458
Epoch 9, val loss: 4.51274, train loss: 0.26686
Epoch 9, age val loss: 1.27876, gender val loss: 0.00000, ethnicity val loss: 3.23398

--------------------------------- Prune 60 % ---------------------------------
---------------- Train Initial Model ----------------
Epoch 0, val loss: inf -> 2.01535, train loss: 0.94282
Epoch 0, age val loss: 1.07206, gender val loss: 0.34978, ethnicity val loss: 0.59351
Epoch 1, val loss: 2.30169, train loss: 0.71277
Epoch 1, age val loss: 0.92388, gender val loss: 0.53687, ethnicity val loss: 0.84094
Epoch 2, val loss: 2.39538, train loss: 0.44039
Epoch 2, age val loss: 0.67605, gender val loss: 0.69012, ethnicity val loss: 1.02922
Epoch 3, val loss: 4.37105, train loss: 0.16469
Epoch 3, age val loss: 0.42323, gender val loss: 2.40319, ethnicity val loss: 1.54464
Epoch 4, val loss: 7.20776, train loss: 0.03966
Epoch 4


--------------------------------- Prune 90 % ---------------------------------
---------------- Train Initial Model ----------------
Epoch 0, val loss: inf -> 2.31615, train loss: 0.93626
Epoch 0, age val loss: 1.07166, gender val loss: 0.68136, ethnicity val loss: 0.56313
Epoch 1, val loss: 2.84280, train loss: 0.71894
Epoch 1, age val loss: 0.91880, gender val loss: 1.21352, ethnicity val loss: 0.71047
Epoch 2, val loss: 2.44498, train loss: 0.47902
Epoch 2, age val loss: 0.56591, gender val loss: 0.87555, ethnicity val loss: 1.00353
Epoch 3, val loss: 2.79011, train loss: 0.17669
Epoch 3, age val loss: 0.34622, gender val loss: 1.26489, ethnicity val loss: 1.17900
Epoch 4, val loss: 2.31615 -> 1.73619, train loss: 0.06973
Epoch 4, age val loss: 0.24623, gender val loss: 0.16007, ethnicity val loss: 1.32988
Epoch 5, val loss: 3.22625, train loss: 0.14390
Epoch 5, age val loss: 0.44217, gender val loss: 0.85186, ethnicity val loss: 1.93222
Epoch 6, val loss: 5.48179, train loss: 0.07

Notes: pruning the linear task-specific layers was sometimes causing an index-out-of-bounds error, which I think is due to it pruning off the final layer. As a fix, trying ignoring the last layer when pruning the linear layers. Note: once I edited the linear layer pruning to not prune the last layer, this problem was resolved.

However, pruning the linear layer caused more drastic performance decreases, so changed it to only prune the convolutional layers.

Note: pruning causes poor performance. Things to try: reset optimizer before pruning. This worked

Notes about this notebook. The MTL notebook generates models by first training the multitask model, pruning it, and finetuning it either for MTL or for specific tasks. The other notebook, SingleTask, does not use the MTL model or objtective at all.

#### Age

In [7]:
task = 'age'
num_epochs = 10
            
for i in range(10):
    print(f'--------------------------------- Prune {i*10} % ---------------------------------')
    prune_pct = 1.0*i / 10
    scores, mean_lat, std_lat = pruned_mtl_training(task, prune_pct, num_epochs, train_loader, val_loader, val_dataset)
    print()

--------------------------------- Prune 0 % ---------------------------------
---------------- Train Initial Model ----------------
Epoch 0, val loss: inf -> 1.95351, train loss: 0.95441
Epoch 0, age val loss: 1.06501, gender val loss: 0.31159, ethnicity val loss: 0.57691
Epoch 1, val loss: 2.08729, train loss: 0.74023
Epoch 1, age val loss: 0.89252, gender val loss: 0.37914, ethnicity val loss: 0.81563
Epoch 2, val loss: 2.14401, train loss: 0.47167
Epoch 2, age val loss: 0.62423, gender val loss: 0.23075, ethnicity val loss: 1.28903
Epoch 3, val loss: 2.83230, train loss: 0.20065
Epoch 3, age val loss: 0.43552, gender val loss: 0.03310, ethnicity val loss: 2.36368
Epoch 4, val loss: 4.00100, train loss: 0.02613
Epoch 4, age val loss: 0.64045, gender val loss: 0.00037, ethnicity val loss: 3.36018
Epoch 5, val loss: 3.83853, train loss: 0.22754
Epoch 5, age val loss: 0.37553, gender val loss: 0.00003, ethnicity val loss: 3.46297
Epoch 6, val loss: 3.41531, train loss: 0.06240
Epoch 6, 

Epoch 8, val loss: 126.99056, train loss: 11.62866
Epoch 9, val loss: 122.70910, train loss: 5.43422

--------------------------------- Prune 40 % ---------------------------------
---------------- Train Initial Model ----------------
Epoch 0, val loss: inf -> 1.85794, train loss: 0.93662
Epoch 0, age val loss: 1.04173, gender val loss: 0.22735, ethnicity val loss: 0.58886
Epoch 1, val loss: 2.28055, train loss: 0.77383
Epoch 1, age val loss: 0.93842, gender val loss: 0.75119, ethnicity val loss: 0.59094
Epoch 2, val loss: 2.01551, train loss: 0.49779
Epoch 2, age val loss: 0.66322, gender val loss: 0.61048, ethnicity val loss: 0.74180
Epoch 3, val loss: 2.69480, train loss: 0.23962
Epoch 3, age val loss: 0.37127, gender val loss: 1.14608, ethnicity val loss: 1.17745
Epoch 4, val loss: 4.09589, train loss: 0.05328
Epoch 4, age val loss: 0.74311, gender val loss: 1.48202, ethnicity val loss: 1.87077
Epoch 5, val loss: 2.75898, train loss: 0.11895
Epoch 5, age val loss: 0.52745, gender v

Epoch 7, val loss: 246.39484 -> 246.22522, train loss: 148.03574
Epoch 8, val loss: 251.50934, train loss: 144.16738
Epoch 9, val loss: 257.27675, train loss: 140.12454

--------------------------------- Prune 80 % ---------------------------------
---------------- Train Initial Model ----------------
Epoch 0, val loss: inf -> 1.82011, train loss: 0.94662
Epoch 0, age val loss: 0.97237, gender val loss: 0.24685, ethnicity val loss: 0.60090
Epoch 1, val loss: 1.82011 -> 1.62019, train loss: 0.73697
Epoch 1, age val loss: 0.84348, gender val loss: 0.06047, ethnicity val loss: 0.71624
Epoch 2, val loss: 1.62019 -> 1.53536, train loss: 0.41041
Epoch 2, age val loss: 0.60938, gender val loss: 0.00768, ethnicity val loss: 0.91830
Epoch 3, val loss: 1.69515, train loss: 0.13594
Epoch 3, age val loss: 0.29634, gender val loss: 0.00095, ethnicity val loss: 1.39786
Epoch 4, val loss: 2.03781, train loss: 0.08632
Epoch 4, age val loss: 0.28982, gender val loss: 0.00017, ethnicity val loss: 1.7478

#### Gender

In [8]:
task = 'gender'
num_epochs = 10

for i in range(10):
    print(f'--------------------------------- Prune {i*10} % ---------------------------------')
    prune_pct = 1.0*i / 10
    scores, mean_lat, std_lat = pruned_mtl_training(task, prune_pct, num_epochs, train_loader, val_loader, val_dataset)
    print()

--------------------------------- Prune 0 % ---------------------------------
---------------- Train Initial Model ----------------
Epoch 0, val loss: inf -> 2.08465, train loss: 0.95042
Epoch 0, age val loss: 1.05347, gender val loss: 0.52194, ethnicity val loss: 0.50924
Epoch 1, val loss: 2.08465 -> 1.78844, train loss: 0.72048
Epoch 1, age val loss: 0.74266, gender val loss: 0.04444, ethnicity val loss: 1.00135
Epoch 2, val loss: 1.92981, train loss: 0.47543
Epoch 2, age val loss: 0.46200, gender val loss: 0.04203, ethnicity val loss: 1.42578
Epoch 3, val loss: 2.07263, train loss: 0.22221
Epoch 3, age val loss: 0.37760, gender val loss: 0.04082, ethnicity val loss: 1.65422
Epoch 4, val loss: 2.18199, train loss: 0.04281
Epoch 4, age val loss: 0.45507, gender val loss: 0.08963, ethnicity val loss: 1.63730
Epoch 5, val loss: 2.18980, train loss: 0.10237
Epoch 5, age val loss: 0.44892, gender val loss: 0.01201, ethnicity val loss: 1.72888
Epoch 6, val loss: 2.26791, train loss: 0.0921

---------------- Train Initial Model ----------------
Epoch 0, val loss: inf -> 1.81610, train loss: 0.94212
Epoch 0, age val loss: 1.02467, gender val loss: 0.10281, ethnicity val loss: 0.68862
Epoch 1, val loss: 1.96029, train loss: 0.77842
Epoch 1, age val loss: 0.87062, gender val loss: 0.34176, ethnicity val loss: 0.74790
Epoch 2, val loss: 3.42140, train loss: 0.49232
Epoch 2, age val loss: 0.71333, gender val loss: 1.92529, ethnicity val loss: 0.78279
Epoch 3, val loss: 5.55965, train loss: 0.25600
Epoch 3, age val loss: 0.31761, gender val loss: 4.39649, ethnicity val loss: 0.84555
Epoch 4, val loss: 9.38105, train loss: 0.06336
Epoch 4, age val loss: 0.80653, gender val loss: 7.59237, ethnicity val loss: 0.98214
Epoch 5, val loss: 10.62627, train loss: 0.12635
Epoch 5, age val loss: 0.70316, gender val loss: 8.84233, ethnicity val loss: 1.08077
Epoch 6, val loss: 10.90054, train loss: 0.08068
Epoch 6, age val loss: 0.44883, gender val loss: 9.29486, ethnicity val loss: 1.15685


--------------------------------- Prune 80 % ---------------------------------
---------------- Train Initial Model ----------------
Epoch 0, val loss: inf -> 1.84363, train loss: 0.95203
Epoch 0, age val loss: 1.06241, gender val loss: 0.24880, ethnicity val loss: 0.53241
Epoch 1, val loss: 1.93327, train loss: 0.81238
Epoch 1, age val loss: 0.97820, gender val loss: 0.32382, ethnicity val loss: 0.63126
Epoch 2, val loss: 2.15490, train loss: 0.48022
Epoch 2, age val loss: 0.78591, gender val loss: 0.29044, ethnicity val loss: 1.07855
Epoch 3, val loss: 2.33259, train loss: 0.20027
Epoch 3, age val loss: 0.62198, gender val loss: 0.24176, ethnicity val loss: 1.46885
Epoch 4, val loss: 2.64172, train loss: 0.01659
Epoch 4, age val loss: 0.63440, gender val loss: 0.04947, ethnicity val loss: 1.95785
Epoch 5, val loss: 2.22200, train loss: 0.22268
Epoch 5, age val loss: 0.51623, gender val loss: 0.02900, ethnicity val loss: 1.67676
Epoch 6, val loss: 2.00131, train loss: 0.08457
Epoch 6

#### Ethnicity

In [9]:
task = 'ethnicity'
num_epochs = 10
            
for i in range(10):
    print(f'--------------------------------- Prune {i*10} % ---------------------------------')
    prune_pct = 1.0*i / 10
    scores, mean_lat, std_lat = pruned_mtl_training(task, prune_pct, num_epochs, train_loader, val_loader, val_dataset)
    print()

--------------------------------- Prune 0 % ---------------------------------
---------------- Train Initial Model ----------------
Epoch 0, val loss: inf -> 2.02071, train loss: 0.95567
Epoch 0, age val loss: 1.06505, gender val loss: 0.30760, ethnicity val loss: 0.64806
Epoch 1, val loss: 2.09033, train loss: 0.74389
Epoch 1, age val loss: 0.90041, gender val loss: 0.18997, ethnicity val loss: 0.99995
Epoch 2, val loss: 2.02071 -> 1.93336, train loss: 0.49921
Epoch 2, age val loss: 0.74764, gender val loss: 0.01381, ethnicity val loss: 1.17190
Epoch 3, val loss: 1.93336 -> 1.89175, train loss: 0.27554
Epoch 3, age val loss: 0.50183, gender val loss: 0.00009, ethnicity val loss: 1.38982
Epoch 4, val loss: 2.17449, train loss: 0.05836
Epoch 4, age val loss: 0.39146, gender val loss: 0.00000, ethnicity val loss: 1.78303
Epoch 5, val loss: 2.62200, train loss: 0.10490
Epoch 5, age val loss: 0.40774, gender val loss: 0.00000, ethnicity val loss: 2.21426
Epoch 6, val loss: 2.92037, train l

---------------- Train Initial Model ----------------
Epoch 0, val loss: inf -> 1.81074, train loss: 0.95184
Epoch 0, age val loss: 1.07847, gender val loss: 0.13809, ethnicity val loss: 0.59419
Epoch 1, val loss: 1.95933, train loss: 0.76145
Epoch 1, age val loss: 0.90627, gender val loss: 0.17195, ethnicity val loss: 0.88111
Epoch 2, val loss: 1.89352, train loss: 0.52694
Epoch 2, age val loss: 0.63583, gender val loss: 0.08155, ethnicity val loss: 1.17614
Epoch 3, val loss: 2.04859, train loss: 0.28566
Epoch 3, age val loss: 0.44601, gender val loss: 0.19343, ethnicity val loss: 1.40915
Epoch 4, val loss: 2.62822, train loss: 0.04970
Epoch 4, age val loss: 0.68537, gender val loss: 0.05998, ethnicity val loss: 1.88287
Epoch 5, val loss: 2.94018, train loss: 0.11053
Epoch 5, age val loss: 0.70628, gender val loss: 0.00158, ethnicity val loss: 2.23233
Epoch 6, val loss: 3.20117, train loss: 0.15169
Epoch 6, age val loss: 0.59713, gender val loss: 0.00010, ethnicity val loss: 2.60394
E

Epoch 3, val loss: 1.90013, train loss: 0.31732
Epoch 3, age val loss: 0.71795, gender val loss: 0.50999, ethnicity val loss: 0.67219
Epoch 4, val loss: 1.89591 -> 1.74695, train loss: 0.09097
Epoch 4, age val loss: 0.42715, gender val loss: 0.57056, ethnicity val loss: 0.74924
Epoch 5, val loss: 2.06891, train loss: 0.09112
Epoch 5, age val loss: 0.20394, gender val loss: 0.94599, ethnicity val loss: 0.91898
Epoch 6, val loss: 2.37101, train loss: 0.09432
Epoch 6, age val loss: 0.23101, gender val loss: 1.05760, ethnicity val loss: 1.08240
Epoch 7, val loss: 2.14568, train loss: 0.04577
Epoch 7, age val loss: 0.30133, gender val loss: 0.78944, ethnicity val loss: 1.05491
Epoch 8, val loss: 1.77417, train loss: 0.02470
Epoch 8, age val loss: 0.35417, gender val loss: 0.27486, ethnicity val loss: 1.14514
Epoch 9, val loss: 2.47391, train loss: 0.03503
Epoch 9, age val loss: 0.35598, gender val loss: 0.73581, ethnicity val loss: 1.38213
-------------- Fine-tuning Pruned Model -----------